In [9]:
'''Methods Validation'''

import sys, os, pathlib

# directory corrente del notebook
NOTEBOOK_DIR = pathlib.Path(os.getcwd())  

# root del progetto = la cartella che contiene "src/"
ROOT = NOTEBOOK_DIR.parents[0]  

sys.path.insert(0, str(ROOT))

from src.Heston_FFT import heston_price_fft
from src.Heston_MC import heston_mc_qe_price
from src.Heston_PINN import (
    HestonPINN, scale_inputs, heston_pde_loss, heston_terminal_loss,
    boundary_S0_loss, boundary_Smax_loss, boundary_v_neumann_loss,
    train_heston
)

In [10]:
import time, numpy as np, torch

# HESTON PARAMS 
S0   = 105.0
K    = 105.0
T    = 1.0
r    = 0.05
q    = 0.0

v0     = 0.04
kappa  = 1.5
theta  = 0.04
sigma_v= 0.3
rho    = -0.7

# FFT PARAMS
alpha  = 1.5
fft_N  = 4096     
eta    = 0.1      

# MC PARAMS
mc_steps = 200
mc_paths = 200_000
seed     = 42

In [11]:
t0 = time.perf_counter()
price_fft = heston_price_fft(S0, K, T, r, v0, kappa, theta, sigma_v, rho,
                             option_type="call", q=q, alpha=alpha, N=fft_N, eta=eta)
t1 = time.perf_counter()
dt_fft = t1 - t0
print(f"FFT:  {price_fft:.6f}  ({dt_fft*1e3:.2f} ms)")


t0 = time.perf_counter()
price_mc, stderr_mc = heston_mc_qe_price(S0, K, T, r, q, v0, kappa, theta, sigma_v, rho,
                                         steps=mc_steps, paths=mc_paths,
                                         antithetic=True, seed=seed, option_type="call")
t1 = time.perf_counter()
dt_mc = t1 - t0
print(f"MC  :  {price_mc:.6f}  ± {1.96*stderr_mc:.6f} (95% CI)  ({dt_mc:.2f} s)")

FFT:  10.879962  (8.99 ms)
MC  :  7.405017  ± 0.022117 (95% CI)  (1.36 s)


In [4]:
'''PINN Train'''

# costruisci e allena "rapido" (poi farai run piu' lunghi)
model = HestonPINN()
_ = train_heston(model,
                 epochs=3000, lr=1e-3,
                 K=K, T=T, S_max=4*S0, v_max=0.8,
                 r=r, q=q, rho=rho, kappa=kappa, theta=theta, sigma=sigma_v,
                 batch_pde=4096, seed=0)

# inference PINN a t=0, S=S0, v=v0
def pinn_price_at_spot(model, S, K, T, v0):
    model.eval()
    with torch.no_grad():
        S_t = torch.tensor([S], dtype=torch.float64)
        v_t = torch.tensor([v0], dtype=torch.float64)
        t_t = torch.tensor([0.0], dtype=torch.float64)
        x = scale_inputs(S_t, v_t, t_t, K=K, T=T, v_max=0.8)
        V = model(x).item()
    return V

t0 = time.perf_counter()
price_pinn = pinn_price_at_spot(model, S0, K, T, v0)
t1 = time.perf_counter()
dt_pinn = t1 - t0
print(f"PINN: {price_pinn:.6f}  ({dt_pinn*1e3:.3f} ms)")

[ep 0] total=2.5574e+05 | pde=4.1713e-03 T=1.5497e+04 S0=5.6513e-03 Smax=1.0077e+05
[ep 500] total=1.8958e+05 | pde=7.3458e+01 T=1.1049e+04 S0=1.0611e+00 Smax=7.9017e+04
[ep 1000] total=1.4100e+05 | pde=1.1107e+02 T=7.9140e+03 S0=1.8493e-01 Smax=6.1743e+04
[ep 1500] total=9.5583e+04 | pde=1.1872e+02 T=4.8788e+03 S0=7.5368e-02 Smax=4.6665e+04
[ep 2000] total=6.7274e+04 | pde=2.9498e+02 T=3.3267e+03 S0=1.4774e-02 Smax=3.3702e+04
[ep 2500] total=4.2591e+04 | pde=4.1920e+02 T=1.9103e+03 S0=3.0279e-02 Smax=2.3063e+04
PINN: 13.729799  (2.740 ms)


**RESULTS**

In [13]:
def bps(x): return 1e4 * x  # 1 bp = 1e-4

err_mc_bps   = bps(abs(price_mc   - price_fft) / max(1.0, price_fft))
err_pinn_bps = bps(abs(price_pinn - price_fft) / max(1.0, price_fft))

print(f"\nPrices: FFT={price_fft:.3f}  |  MC={price_mc:.3f}  |  PINN = {price_pinn:.3f}")
print(f"Errors vs FFT (bps): MC={err_mc_bps:.2f}  |  PINN={err_pinn_bps:.2f}")
print(f"Time: FFT={dt_fft*1e3:.2f} ms  |  MC={dt_mc:.2f} s  |  PINN {dt_pinn*1e3:.2f} ms")


Prices: FFT=10.880  |  MC=7.405  |  PINN = 12.126
Errors vs FFT (bps): MC=3193.89  |  PINN=1145.39
Time: FFT=34.67 ms  |  MC=2.75 s  |  PINN 4.47 ms
